In [17]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pprint


service = EdgeService(executable_path=r"..\edgedriver_win64\msedgedriver.exe")
browser = webdriver.Edge(service=service)


In [18]:
browser.get("https://sellercenter.daraz.com.np/apps/seller/login")

In [ ]:

phone_input = browser.find_element(By.XPATH, '//*[@id="account"]')
phone_input.send_keys('9811400555')

password_input = browser.find_element(By.XPATH, '//*[@id="password"]')
password_input.send_keys('Alam@5555')

login_button = browser.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div[1]/div/div[2]/div/div/div[2]/form/div[3]/button')
login_button.click()

In [3]:
def required_attribute(string):
    # Split the input string into lines
    lines = string.split('\n')

    # Extract elements that have '*' before them
    extracted_elements = [lines[i + 1].strip() for i in range(len(lines) - 1) if lines[i].strip() == '*']
    return extracted_elements

In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
import time

def mandatory__attribute_name(browser,wait):

    wait = WebDriverWait(browser, 10)
    element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/section/div[2]/div/div[1]/div/div/div[2]/form/div/div[1]/div[2]/div/div/div/div[2]/div[1]/div[2]/button')))

    # Scroll the element into view and allow some time for the scrolling
    browser.execute_script("arguments[0].scrollIntoView(true);", element)
    time.sleep(2)

    # Try clicking with exception handling for intercepted clicks
    try:
        element.click()
        time.sleep(2)  # Extra wait after the click
    except ElementClickInterceptedException:
        print("Element click intercepted, retrying with JavaScript click.")
        # Retry the click using JavaScript if intercepted
        browser.execute_script("arguments[0].click();", element)
        time.sleep(2)
     # Allow time for the UI to update

    # Locate the grid containing attribute options
    grid = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/section/div[2]/div/div[1]/div/div/div[2]/form/div/div[1]/div[2]/div/div/div/div[2]/div[1]/div[1]/div")))
    all_fields=grid.text
    mandatory_attribute=required_attribute(all_fields)

    # Append attribute details to the list
    return mandatory_attribute

In [6]:
import json

# Path to your JSON file
file_path = 'categories_reference.json'  # Adjust the path if necessary

# Load the JSON data
with open(file_path, 'r') as file:
    data = json.load(file)   

paths = []

def find_paths(d, current_path=None):
    if current_path is None:
        current_path = []
    
    for k, v in d.items():
        new_path = current_path + [k]  # Create a new path as a list
        if isinstance(v, dict) and v:  # Check if the value is a non-empty dict
            find_paths(v, new_path)
        else:  # Leaf node (empty dict)
            paths.append(new_path)   

In [7]:
find_paths(data)
path_str=[' > '.join(path) for path in paths]

In [8]:
mandatory__attribute_list=[]

In [10]:
wait = WebDriverWait(browser, 10)
browser.refresh()
time.sleep(5)
for path in path_str:
    print(f'\nProcessing {path_str.index(path)}: {path}', end="  ")
    if "test" not in path.lower():
        try:
            # Wait for the category dropdown to be clickable
            category_dropdown = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Please select category or search with keyword']")))
            category_dropdown.click()
            time.sleep(3)


            # Click on the search field to search categories
            search_category = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@placeholder="Search category"]')))
            search_category.click()
            time.sleep(3)
            search_category.send_keys(path)
            time.sleep(2)

            # Select the first category from the dropdown list
            first_cat_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.next-overlay-wrapper.opened > div > div > div.gc-category-cascade-view.aplus-module-auto-exp > div > div.category-search-input.aplus-auto-exp > div ul li:first-child")))
            first_cat_select.click()
            time.sleep(3)

            # Confirm category selection
            confirm_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-spm='d_category_selector_confirm_btn']")))
            confirm_button.click()
            time.sleep(5)
            mandatory_attribute=mandatory__attribute_name(browser,wait)
            mandatory__attribute_list.append({"name":path,
            "mandatory_attribute":mandatory_attribute})
            print(mandatory_attribute)
            browser.refresh()
            time.sleep(5)
        except Exception as e:
            # Handle errors and continue with the next category
            print(f"Error processing category: {path} \n")
            mandatory__attribute_list.append({"name":path,
            "mandatory_attribute":[]})
            browser.refresh()
            time.sleep(5)



Processing 0: Test Product （wont be seen in seller pls dont inactive) > PV merge test category > v merge in same P - SKU  
Processing 1: Test Product （wont be seen in seller pls dont inactive) > PV merge test category > v merge in same P - SPU  
Processing 2: Test Product （wont be seen in seller pls dont inactive) > PV merge test category > v merge cross P - SKU  
Processing 3: Test Product （wont be seen in seller pls dont inactive) > PV merge test category > v merge cross P - SPU  
Processing 4: Test Product （wont be seen in seller pls dont inactive) > PV merge test category > P and PV merge - SKU  
Processing 5: Test Product （wont be seen in seller pls dont inactive) > PV merge test category > P and PV merge - SPU  
Processing 6: Test Product （wont be seen in seller pls dont inactive) > PV merge test category > P merge - SKU  
Processing 7: Test Product （wont be seen in seller pls dont inactive) > PV merge test category > P merge - SPU  
Processing 8: Test Product （wont be seen in s

KeyboardInterrupt: 

For Failed Items

In [19]:
failed_mandatory_attribute_name=[[mandatory__attribute_list.index(i),i] for i in mandatory__attribute_list if len(i['mandatory_attribute'])==0]

In [20]:
len(failed_mandatory_attribute_name)

16

In [21]:
wait = WebDriverWait(browser, 10)
browser.refresh()
time.sleep(5)
for path in failed_mandatory_attribute_name:
    print(f'Processing attribute {failed_mandatory_attribute_name.index(path)}:\nAttribute_item_index= {mandatory__attribute_list.index(path[1])}, {path[1]['name']}')
    if "test" not in path[1]['name'].lower():
        try:
        # Wait for the category dropdown to be clickable
            category_dropdown = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Please select category or search with keyword']")))
            category_dropdown.click()
            time.sleep(3)


            # Click on the search field to search categories
            search_category = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@placeholder="Search category"]')))
            search_category.click()
            time.sleep(3)
            search_category.send_keys(path[1]['name'])
            time.sleep(2)

            # Select the first category from the dropdown list
            first_cat_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.next-overlay-wrapper.opened > div > div > div.gc-category-cascade-view.aplus-module-auto-exp > div > div.category-search-input.aplus-auto-exp > div ul li:first-child")))
            first_cat_select.click()
            time.sleep(3)

            # Confirm category selection
            confirm_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-spm='d_category_selector_confirm_btn']")))
            confirm_button.click()
            time.sleep(5)
            mandatory_attribute=mandatory__attribute_name(browser,wait)

            mandatory__attribute_list[mandatory__attribute_list.index(path[1])]['mandatory_attribute']=mandatory_attribute
            browser.refresh()
            time.sleep(5)
        except Exception as e:
            # Handle errors and continue with the next category
            print(f"Error processing category: {path}")
            browser.refresh()
            time.sleep(5)

Processing attribute 0:
Attribute_item_index= 138, Computers & Laptops > Monitors
Error processing category: [138, {'name': 'Computers & Laptops > Monitors', 'mandatory_attribute': []}]
Processing attribute 1:
Attribute_item_index= 515, Health & Beauty > Food Supplements > Weight Management > Appetite Suppressant
Error processing category: [515, {'name': 'Health & Beauty > Food Supplements > Weight Management > Appetite Suppressant', 'mandatory_attribute': []}]
Processing attribute 2:
Attribute_item_index= 516, Health & Beauty > Food Supplements > Weight Management > Carb Blockers
Error processing category: [516, {'name': 'Health & Beauty > Food Supplements > Weight Management > Carb Blockers', 'mandatory_attribute': []}]
Processing attribute 3:
Attribute_item_index= 518, Health & Beauty > Food Supplements > Weight Management > Gifts & Value Sets
Error processing category: [518, {'name': 'Health & Beauty > Food Supplements > Weight Management > Gifts & Value Sets', 'mandatory_attribute

Export

In [22]:
import json
with open('mandatory_attribute.json', 'w') as json_file:
    json.dump(mandatory__attribute_list, json_file, indent=4)